In [1]:
from pyspark.sql import SparkSession
import pyspark.pandas as ps

/Users/nakayamadaiki/.asdf/installs/python/3.11.6/lib/python3.11/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [6]:
# Loading dataset
spark = SparkSession.builder \
    .appName("SocialMediaAddictionPredictor") \
    .getOrCreate()
print("Reading the dataset...")
spark_df = spark.read.csv('./Data/dataset.csv')

24/04/16 18:37:31 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Reading the dataset...


/Users/nakayamadaiki/.asdf/installs/python/3.11.6/lib/python3.11/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [ ]:
spark_df.rename(columns={'1. What is your age?': 'Age', '2. Gender': 'Gender', '3. Relationship Status': 'Relationship Status',
                         '4. Occupation Status': 'Occupation',
                         '5. What type of organizations are you affiliated with?': 'Affiliations',
                         '6. Do you use social media?': 'Social Media User?',
                         '7. What social media platforms do you commonly use?': 'Platforms Used',
                         '8. What is the average time you spend on social media every day?': 'Hours Per Day',
                         '9. How often do you find yourself using Social media without a specific purpose?': 'ADHD Q1',
                         '10. How often do you get distracted by Social media when you are busy doing something?': 'ADHD Q2',
                         "11. Do you feel restless if you haven't used Social media in a while?": 'Anxiety Q1',
                         '12. On a scale of 1 to 5, how easily distracted are you?': 'ADHD Q3',
                         '13. On a scale of 1 to 5, how much are you bothered by worries?': 'Anxiety Q2',
                         '14. Do you find it difficult to concentrate on things?': 'ADHD Q4',
                         '15. On a scale of 1-5, how often do you compare yourself to other successful people through the use of social media?': 'Self Esteem Q1',
                         '16. Following the previous question, how do you feel about these comparisons, generally speaking?': 'Self Esteem Q2',
                         '17. How often do you look to seek validation from features of social media?': 'Self Esteem Q3',
                         '18. How often do you feel depressed or down?': 'Depression Q1',
                         '19. On a scale of 1 to 5, how frequently does your interest in daily activities fluctuate?': 'Depression Q2',
                         '20. On a scale of 1 to 5, how often do you face issues regarding sleep?': 'Depression Q3'}, inplace=True)

24/04/17 04:50:41 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 3645171 ms exceeds timeout 120000 ms
24/04/17 04:50:42 WARN SparkContext: Killing executors is not supported by current scheduler.
24/04/17 04:50:42 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at 

In [ ]:
# Define causal model
model = CausalModel(
    data=df,
    treatment='Total Score',
    outcome='Hours Per Day',
    common_causes=['ADHD Score', 'Anxiety Score',
                   'Self Esteem Score', 'Depression Score']
)

# Identify causal effect
identified_estimand = model.identify_effect()

# Estimate causal effect
estimate = model.estimate_effect(identified_estimand,
                                 method_name="backdoor.linear_regression")

print(estimate)

INFO:dowhy.causal_model:Model to find the causal effect of treatment ['Total Score'] on outcome ['Hours Per Day']
INFO:dowhy.causal_identifier.auto_identifier:Causal effect can be identified.
INFO:dowhy.causal_identifier.auto_identifier:Instrumental variables for treatment and outcome:[]
INFO:dowhy.causal_identifier.auto_identifier:Frontdoor variables for treatment and outcome:[]
INFO:dowhy.causal_model:linear_regression
INFO:dowhy.causal_estimator:INFO: Using Linear Regression Estimator
INFO:dowhy.causal_estimator:b: Hours Per Day~Total Score+Depression Score+ADHD Score+Anxiety Score+Self Esteem Score


*** Causal Estimate ***

## Identified estimand
Estimand type: EstimandType.NONPARAMETRIC_ATE

### Estimand : 1
Estimand name: backdoor
Estimand expression:
      d                                                                       
──────────────(E[Hours Per Day|Depression Score,ADHD Score,Anxiety Score,Self 
d[Total Score]                                                                

              
Esteem Score])
              
Estimand assumption 1, Unconfoundedness: If U→{Total Score} and U→Hours Per Day then P(Hours Per Day|Total Score,Depression Score,ADHD Score,Anxiety Score,Self Esteem Score,U) = P(Hours Per Day|Total Score,Depression Score,ADHD Score,Anxiety Score,Self Esteem Score)

## Realized estimand
b: Hours Per Day~Total Score+Depression Score+ADHD Score+Anxiety Score+Self Esteem Score
Target units: ate

## Estimate
Mean value: 0.0705858818267544



/Users/nakayamadaiki/.asdf/installs/python/3.11.6/lib/python3.11/site-packages/dowhy/causal_estimators/regression_estimator.py:131: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept_parameter = self.model.params[0]
